In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [ ]:
#cleaning data
df = pd.read_csv('2017-18 OHL Skaters.csv')
df = df.drop(columns=['#'])
stats = ['G/GP','A1/GP','A2/GP','P/GP','Sh/GP']

#min games played
df = df[df.GP >= 30]

df.head(5)

In [ ]:
# clustering algorithm
k_means = KMeans(init='k-means++', n_clusters=8, n_init=100)
k_means.fit(preprocessing.scale(df[stats]))

In [ ]:
# add cluster to df
df['cluster'] = k_means.labels_

# get cluster centroids
centers = k_means.cluster_centers_
center_df = pd.DataFrame(centers, columns=stats)

# get the mean values for each stat
means = []
for col in range(centers.shape[1]):
    means.append(centers[:,col].mean())

# find values relative to mean
rel_values = centers - np.array(means)
rel_df = pd.DataFrame(rel_values, columns=stats)

center_df.head(5)

In [ ]:
# print results
print("\nK-Means Clustering of OHL Players:\n")
for i in range(0,len(df.cluster.value_counts())):
    print('Group:', i)
    print('{stat:11}{val:7}{rank}'.format(stat='Stat', val='Value', rank='Rank'))
    for j in range(len(stats)):
        print('{stat:8}: {val:6.2f}  ({rank:.0f}/{of})'.format(stat=stats[j],
                                         rank=rel_df.rank(ascending=False).iloc[i].values[j],
                                         val=rel_df.iloc[i].values[j],
                                         of=len(rel_df)))
    print('\nTypical players:')
    for j in range(5):
        print('-',df[df.cluster==i]['Name'].head().values[j])
    print('--------------------\n')

Next Steps:
1. figure out how to sort by distance from centroid and list those players, right now its just random
2. plug in better data, right now it's just point production related
3. make two models for forwards vs defencement
4. Make model using NHL data for CUC